# Notebook del Curso completo de NLP Parte 2

Link al video de youtube:  
https://youtu.be/9x1QtYNLJRY?si=xX6LYMMtcutlw5En&t=3234


# Ejercicio integrador


Es este ejercicio vamos a aplicar los visto en el notebook 1, mediante el dataset importado de kaggle vamosa  entrenar un modelo con texto en español, definiendo las variables de noticias y tipo de noticia para luego realizar predicciones.

## Importación e instalación de librerías

In [ ]:
!pip install -U scikit-learn
!python -m spacy download es_core_news_sm -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 14.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 41.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Importar archivo del dataset de kaggle

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kevinmorgado/spanish-news-classification")

print("Path to dataset files:", path)

100%|██████████| 1.38M/1.38M [00:00<00:00, 30.7MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/kevinmorgado/spanish-news-classification/versions/1


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(path+'/df_total.csv', encoding='UTF-8')
df

,url,news,Type
0,https://www.larepublica.co/redirect/post/3201905,Durante el foro La banca articulador empresari...,Otra
1,https://www.larepublica.co/redirect/post/3210288,El regulador de valores de China dijo el domin...,Regulaciones
2,https://www.larepublica.co/redirect/post/3240676,En una industria históricamente masculina como...,Alianzas
3,https://www.larepublica.co/redirect/post/3342889,Con el dato de marzo el IPC interanual encaden...,Macroeconomia
4,https://www.larepublica.co/redirect/post/3427208,Ayer en Cartagena se dio inicio a la versión n...,Otra
...,...,...,...
1212,https://www.bbva.com/es/como-lograr-que-los-in...,En la vida de toda empresa emergente llega un ...,Innovacion
1213,https://www.bbva.com/es/podcast-como-nos-afect...,La espiral alcista de los precios continúa y g...,Macroeconomia
1214,https://www.larepublica.co/redirect/post/3253735,Las grandes derrotas nacionales son experienci...,Alianzas
1215,https://www.bbva.com/es/bbva-y-barcelona-healt...,BBVA ha alcanzado un acuerdo de colaboración c...,Innovacion


In [ ]:
print("Type:", df["Type"][3])
print("news:", df["news"][3])

Type: Macroeconomia
news: Con el dato de marzo el IPC interanual encadena su decimoquinta tasa positiva consecutiva. La inflación publicada por el INE se ha mantenido igual respecto al avance del 30 de marzo y se sitúa 22 puntos por encima del dato de febrero que ascendió al 76.De esos 22 puntos de diferencia la mayor parte la colocó el grupo de la vivienda 09 puntos por la subida de la electricidad y el del transporte 07 puntos por el alza de los carburantes. También impulsaron el IPC de marzo el aumento de los precios de la restauración y los servicios de alojamiento y al encarecimiento generalizado de los alimentos especialmente del pescado y el marisco de la carne de las legumbres y hortalizas y de la leche el queso y los huevos.Sin tener en cuenta la rebaja del impuesto especial sobre la electricidad y las variaciones sobre otros impuestos el IPC interanual alcanzó en marzo 107 nueve décimas más que la tasa general del 98. Así lo refleja el IPC a impuestos constantes que el INE ta

## Entrenamiento del modelo

In [ ]:
from sklearn.model_selection import train_test_split

X = df["news"]
y = df["Type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_test_split(X, y, test_size=0.2)

El parámetro de test_size define el tamaño de los conjuntos de entrenamiento(train) y prueba(test).

random_state: esto permite la reproducibilidad del experimento, así cuando se repita se pueden obtener los mismos resultados.

## Vectorización de Texto
Convierte el texto de los conjuntos en una matriz de tokens

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

X_test_transformed_dense = X_train_transformed.toarray()

X_train_transformed

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 233764 stored elements and shape (973, 26512)>

### Dimensionalidad
X_train_transformed es una matriz que posee una dimensionalidad de (973, 26512)

### Entrenamiento y Evaluación del Modelo

El segundo fragmento utiliza los datos vectorizados para entrenar un modelo de clasificación y evaluar su rendimiento.

MultinomialNB: Inicializa el modelo de clasificación. El clasificador Naive Bayes se basa en el Teorema de Bayes y asume independencia entre las características (palabras), lo que lo hace computacionalmente muy rápido.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

model = MultinomialNB()
model.fit(X_train_transformed, y_train)
y_pred = model.predict(X_test_transformed)
accuracy = metrics.accuracy_score(y_test, y_pred)

print("Accuracy normal:",accuracy)

Accuracy normal: 0.7991803278688525


El $79.91\%$ de las veces, el modelo predijo correctamente el tipo de noticia.

### Exactitud

La métrica de Exactitud es la medida de rendimiento más sencilla y fundamental en la clasificación:$$Accuracy = \frac{\text{Número de predicciones correctas}}{\text{Número total de predicciones}}$$


## Stemming
Procedemos a transformar el texto para reducir la dimensionalidad

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('stopwords')
nltk.download('punkt_tab')

stemmer = SnowballStemmer('spanish')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
def tokenize_and_steam(text):
  tokens = word_tokenize(text.lower())
  stems = [stemmer.stem(token) for token in tokens if token.isalpha()]
  return ' '.join(stems)

In [ ]:
df['news_stemmer'] = df['news'].apply(tokenize_and_steam)

In [ ]:
df['news_stemmer'][3]


'con el dat de marz el ipc interanual encaden su decimoquint tas posit consecut la inflacion public por el ine se ha manten igual respect al avanc del de marz y se situ punt por encim del dat de febrer que ascend al esos punt de diferent la mayor part la coloc el grup de la viviend punt por la sub de la electr y el del transport punt por el alza de los carbur tambien impuls el ipc de marz el aument de los preci de la restaur y los servici de aloj y al encarec generaliz de los aliment especial del pesc y el marisc de la carn de las legumbr y hortaliz y de la lech el ques y los ten en cuent la rebaj del impuest especial sobr la electr y las variacion sobr otros impuest el ipc interanual alcanz en marz nuev decim mas que la tas general del asi lo reflej el ipc a impuest constant que el ine tambien public en el marc de esta inflacion subyacent sin aliment no elabor ni product energet aument en marz cuatr decim hast su valor mas alto desd septiembr de de este mod la subyacent se situ mas de

### Entrenamiento con datos transformados

In [ ]:
X = df["news_stemmer"]
y = df["Type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = CountVectorizer()

X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

model = MultinomialNB()
model.fit(X_train_transformed, y_train)

y_pred = model.predict(X_test_transformed)
accuracy_stemm = metrics.accuracy_score(y_test, y_pred)

print("Accuracy de stemm",accuracy_stemm)

Accuracy de stemm 0.8278688524590164


In [ ]:
X_train_transformed

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 198177 stored elements and shape (973, 11905)>

### Dimensionalidad
X_train_transformed es una matriz que posee (973, 11905).
A diferencia de sin tratamiento se redujo la dimensionalidad a mas de la mitad.


## Lemmatization

In [ ]:
import spacy
nlp = spacy.load('es_core_news_sm')

In [ ]:
def lemmatize_text(text):
  doc = nlp(text.lower())
  lemmas = [token.lemma_ for token in doc if token.is_alpha]
  return ' '.join(lemmas)

In [ ]:
df['news_lemma'] = df['news'].apply(lemmatize_text)

In [ ]:
df['news_lemma'][3]

'con el dato de marzo el ipc interanual encadenar su decimoquinto tasa positivo consecutivo el inflación publicado por el ine él haber mantener igual respecto al avance del de marzo y él situar punto por encima del dato de febrero que ascender al ese punto de diferencia el mayor parte él colocar el grupo de el vivienda punto por el subida de el electricidad y el del transporte punto por el alza de el carburante también impulsar el ipc de marzo el aumento de el precio de el restauración y el servicio de alojamiento y al encarecimiento generalizado de el alimento especialmente del pescado y el marisco de el carne de el legumbr y hortaliza y de el leche el queso y el tener en cuenta el rebaja del impuesto especial sobre el electricidad y el variación sobre otro impuesto el ipc interanual alcanzar en marzo nueve décima más que el tasa general del así él reflejar el ipc a impuesto constante que el ine también publicar en el marco de este inflación subyacente sin alimento no elaborado ni pro

El texto transformado cambió algunas características de la columna 'news', como cambiar los artículos de 'la' a 'el', y en el caso de algunos verbos 'encadena' -> 'encadenar'.

### Entrenamiento con datos transformados

In [ ]:
X = df["news_lemma"]
y = df["Type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = CountVectorizer()

X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

model = MultinomialNB()
model.fit(X_train_transformed, y_train)

y_pred = model.predict(X_test_transformed)
accuracy_lemma = metrics.accuracy_score(y_test, y_pred)
print("Accuracy lemma:", accuracy_lemma)


Accuracy lemma: 0.8319672131147541


In [ ]:
X_train_transformed

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 193318 stored elements and shape (973, 16530)>

### Dimensionalidad
X_train_transformed es una matriz que posee una dimensión de (973, 16530).

Utilizando la transformación de lematización se redujo la dimensionalidad original, pero no tanto como en stemming.

## Conclusión

Luego de probar con distintos algortimos y ver la tasa de error y su dimensionalidad, se puede optar por elegir según la característica que se desea, puede ser por el que tiene menor error o el que ocupa menos memoria(menor dimensionalidad).